### **Gender Estimation using MobileNetV1**

### 1. Download dataset
- UTKFace source: https://www.kaggle.com/datasets/jangedoo/utkface-new

In [ ]:
!gdown 1HOiaQcnemJWsYuTEHRpa3jfSFYDex7NB

Downloading...
From: https://drive.google.com/uc?id=1HOiaQcnemJWsYuTEHRpa3jfSFYDex7NB
To: /content/UTKFace.zip
100% 347M/347M [01:06<00:00, 5.21MB/s]


In [ ]:
!unzip -q /content/UTKFace.zip -d data

In [ ]:
# Number of images in UTKFace
!ls data/UTKFace | wc -l

23708


In [ ]:
!ls -U data/UTKFace | head -5

32_0_3_20170119200310531.jpg.chip.jpg
80_0_0_20170117173307368.jpg.chip.jpg
47_0_0_20170104210532204.jpg.chip.jpg
10_0_4_20170103212521420.jpg.chip.jpg
95_1_0_20170110182409918.jpg.chip.jpg


### 2. Import libraries

In [ ]:
from google.colab import files
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime

### 3. Process dataset

In [ ]:
# Image size for mobilenetv1
MODEL_INPUT_IMAGE_SIZE = [128, 128]

# Fraction of the dataset to be split test set
TRAIN_TEST_SPLIT = 0.3

# Trick to one-hot encode the label.
y1 = tf.constant([1. ,0.], dtype='float32') # Male
y2 = tf.constant([0. ,1.], dtype='float32') # Female

In [ ]:
def parse_image(filename):
    # Read the image from the filename and resize it.
    image_raw = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image_raw, channels=3) 
    #image = tf.image.resize(image, MODEL_INPUT_IMAGE_SIZE) / 255
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, MODEL_INPUT_IMAGE_SIZE)

    # Split the filename to get the age and the gender. 
    # Convert the age ( str ) and the gender ( str ) to dtype float32.
    parts = tf.strings.split(tf.strings.split(filename, '/')[2], '_')

    # One-hot encode the label
    gender = tf.strings.to_number(parts[1])
    gender_onehot = (gender * y2) + ((1 - gender) * y1)

    return image, gender_onehot

In [ ]:
# List all the image files in the given directory.
list_ds = tf.data.Dataset.list_files('data/UTKFace/*', shuffle=True)
# Map `parse_image` method to all filenames.
dataset = list_ds.map(parse_image, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
# Create train and test splits of the dataset.
num_examples_in_test_ds = int(dataset.cardinality().numpy() * TRAIN_TEST_SPLIT)

test_ds = dataset.take(num_examples_in_test_ds)
train_ds = dataset.skip(num_examples_in_test_ds)

print('Num examples in train ds {}'.format(train_ds.cardinality()))
print('Num examples in test ds {}'.format(test_ds.cardinality()))

Num examples in train ds 16596
Num examples in test ds 7112


### 4. Build and train MobileNetV1

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = (128, 128)
IMG_SHAPE = IMG_SIZE + (3,) # (128, 128, 3)
train_ds = train_ds.batch(BATCH_SIZE)
test_ds = test_ds.batch(BATCH_SIZE)

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.mobilenet.MobileNet(input_shape=IMG_SHAPE,
                                              include_top=False,
                                              weights="imagenet"
                                              )

base_model.trainable = False

image_batch, label_batch = next(iter(train_ds))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

base_model.summary()

17225924/17225924 [==============================] - 0s 0us/step
(32, 4, 4, 1024)
Model: "mobilenet_1.00_128"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 64, 64, 32)        864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 64, 64, 32)       128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 64, 64, 32)        0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)       288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (N

In [ ]:
inputs = tf.keras.Input(shape=IMG_SHAPE)
x = tf.keras.applications.mobilenet.preprocess_input(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
# outputs = tf.keras.layers.Dense(1)(x)
outputs = tf.keras.layers.Dense(2, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 128, 128, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 128, 128, 3)      0         
 a)                                                              
                                                                 
 mobilenet_1.00_128 (Functio  (None, 4, 4, 1024)       3228864   
 nal)                                                            
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                      

In [ ]:
# Init ModelCheckpoint callback
save_dir_ = 'mobilenetv1_gender'  
save_dir = save_dir_ + '/{epoch:02d}-{val_accuracy:.2f}.h5'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint( 
    save_dir , 
    save_best_only=True , 
    monitor='val_accuracy' , 
    mode='max', 
)

tb_log_name = 'mobilenetv1_gender'
# Init TensorBoard Callback
logdir = os.path.join( "tb_logs" , tb_log_name )
tensorboard_callback = tf.keras.callbacks.TensorBoard( logdir )

# Init Early Stopping callback
early_stopping_callback = tf.keras.callbacks.EarlyStopping( monitor='val_accuracy' , patience=5 )

In [ ]:
# warm up

num_epochs = 1
learning_rate = 0.001

model.compile( 
    loss=tf.keras.losses.categorical_crossentropy ,
    optimizer = tf.keras.optimizers.Adam( learning_rate ) , 
    metrics=[ 'accuracy' ]
)

model.fit( 
    train_ds, 
    epochs=num_epochs,  
    validation_data=test_ds 
    # callbacks=[ checkpoint_callback , tensorboard_callback , early_stopping_callback ]
)

519/519 [==============================] - 35s 57ms/step - loss: 0.4366 - accuracy: 0.8010 - val_loss: 0.3336 - val_accuracy: 0.8580


In [ ]:
# fine-tuning
num_epochs = 30
learning_rate = 0.0001
base_model.trainable = True # unfreeze the mobilenet backbone

model.compile( 
    loss=tf.keras.losses.categorical_crossentropy ,
    optimizer = tf.keras.optimizers.Adam( learning_rate ) , 
    metrics=[ 'accuracy' ]
)

model.fit( 
    train_ds, 
    epochs=num_epochs,  
    validation_data=test_ds,
    callbacks=[ checkpoint_callback , tensorboard_callback , early_stopping_callback ]
)

Epoch 1/30
519/519 [==============================] - 60s 101ms/step - loss: 0.3671 - accuracy: 0.8433 - val_loss: 0.2429 - val_accuracy: 0.8997
Epoch 2/30
519/519 [==============================] - 56s 100ms/step - loss: 0.2408 - accuracy: 0.8987 - val_loss: 0.1955 - val_accuracy: 0.9232
Epoch 3/30
519/519 [==============================] - 56s 100ms/step - loss: 0.2095 - accuracy: 0.9120 - val_loss: 0.2677 - val_accuracy: 0.8777
Epoch 4/30
519/519 [==============================] - 55s 99ms/step - loss: 0.1859 - accuracy: 0.9243 - val_loss: 0.1977 - val_accuracy: 0.9134
Epoch 5/30
519/519 [==============================] - 56s 101ms/step - loss: 0.1766 - accuracy: 0.9293 - val_loss: 0.1518 - val_accuracy: 0.9431
Epoch 6/30
519/519 [==============================] - 55s 99ms/step - loss: 0.1611 - accuracy: 0.9346 - val_loss: 0.1267 - val_accuracy: 0.9530
Epoch 7/30
519/519 [==============================] - 55s 99ms/step - loss: 0.1487 - accuracy: 0.9418 - val_loss: 0.1187 - val_accur

### 6. Save trained models

In [ ]:
!ls mobilenetv1_gender/

01-0.90.h5  06-0.95.h5	11-0.96.h5  17-0.98.h5	29-0.99.h5
02-0.92.h5  07-0.95.h5	12-0.97.h5  22-0.99.h5
05-0.94.h5  09-0.96.h5	13-0.98.h5  26-0.99.h5


In [ ]:
# Save trained models to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp mobilenetv1_gender /content/drive/MyDrive/AgeGenderEstimation/